Modelul este prea mare, executia nu a ajuns pana la final

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random
import pandas as pd


MODEL_NAME = "OpenLLM-Ro/RoLlama2-7b-Instruct-DPO-2025-04-23"



tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

print("Model încărcat:", MODEL_NAME)
print("GPU disponibil:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Mulțumesc — rulezi pe GPU!", torch.cuda.get_device_name(0))


SYSTEM_MSG = (
    "Ești un jurist care redactează cereri judiciare clare, concise şi conforme codului de procedură civilă din România. "
    "Creează text complet în limba română, folosind formule standard juridice precum „Subsemnatul…”, „Pentru aceste motive…”, „Vă rugăm să…”."
)

TEMPLATE = """
Redactează o {tip_cerere} completă în limba română, pe baza următoarelor date:
- Instanță: {instanta}
- Reclamant / Apelant / Recursant: {reclamant}
- Pârât / Intimat: {parat}
- Obiectul cererii: {obiect}
- Temei legal: {temei}
- Probe: {probe}
- Ton: formal și juridic
- Lungime dorită: {lungime} cuvinte
"""

tipuri = ["cerere de chemare în judecată", "cerere de apel", "cerere de recurs"]
instante = ["Judecătoria București", "Tribunalul Cluj", "Curtea de Apel Iași"]
reclamanti = ["subsemnatul A.B.", "apelantul C.D.", "recursantul E.F."]
parati = ["Societatea X S.R.L.", "pârâtul G.H.", "intimatul I.J."]
obiecte = [
    "obligarea pârâtului la plata sumei de 10.000 lei reprezentând daune morale",
    "anularea hotărârii civile nr. 123/2024 a Tribunalului Cluj",
    "modificarea dispozitivului sentinței și respingerea acțiunii ca neîntemeiată"
]
temeiuri = [
    "art. 1349 Cod civil și art. 194 C.proc.civ.",
    "art. 466–480 C.proc.civ.",
    "art. 488 alin. 1 pct. 8 C.proc.civ."
]
probe = [
    "înscrisuri, martori, înregistrări video",
    "contractul anexat, corespondența electronică, facturi",
    "hotărâri anterioare și dovezi de plată"
]
lungimi = [150, 200, 250]

def sample_prompt():
    return TEMPLATE.format(
        tip_cerere=random.choice(tipuri),
        instanta=random.choice(instante),
        reclamant=random.choice(reclamanti),
        parat=random.choice(parati),
        obiect=random.choice(obiecte),
        temei=random.choice(temeiuri),
        probe=random.choice(probe),
        lungime=random.choice(lungimi)
    )

def call_model(system, prompt, temperature=0.3, max_new_tokens=400):
    full_prompt = f"System: {system}\nUser: {prompt}\nAssistant:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


for i in range(1):
    p = sample_prompt()
    print(f"\n=== Prompt {i+1} ===\n{p}")
    out = call_model(SYSTEM_MSG, p)
    print(f"\n--- Cerere generată ---\n{out}\n")
